In [9]:
""" K-Means.

Implement K-Means algorithm with TensorFlow, and apply it to classify
handwritten digit images. This example is using the MNIST database of
handwritten digits as training samples (http://yann.lecun.com/exdb/mnist/).
"""

' K-Means.\n\nImplement K-Means algorithm with TensorFlow, and apply it to classify\nhandwritten digit images. This example is using the MNIST database of\nhandwritten digits as training samples (http://yann.lecun.com/exdb/mnist/).\n'

In [10]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib.factorization import KMeans
from tensorflow.examples.tutorials.mnist import input_data
# Ignore all GPUs, tf random forest does not benefit from it.
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = ""


In [11]:
# Import MNIST data
mnist = input_data.read_data_sets("../data/MNIST_data/",one_hot=True)
full_data_x = mnist.train.images

Extracting ../data/MNIST_data/train-images-idx3-ubyte.gz


Extracting ../data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../data/MNIST_data/t10k-labels-idx1-ubyte.gz


In [12]:
# Parameters
num_steps = 50  # Total steps to train
batch_size = 1024  # The number of samples per batch
k = 25  # The number of clusters
num_classes = 10  # The 10 digits
num_features = 784  # Each image is 28x28 pixels
# Input images
X = tf.placeholder(tf.float32, shape=[None, num_features])
# Labels (for assigning a label to a centroid and testing)
Y = tf.placeholder(tf.float32, shape=[None, num_classes])

# K-Means Parameters
kmeans = KMeans(inputs=X, num_clusters=k, distance_metric='cosine', use_mini_batch=True)


In [14]:
# Build KMeans graph
(all_scores, cluster_idx, scores, cluster_centers_initialized,
 init_op, train_op) = kmeans.training_graph()
cluster_idx = cluster_idx[0]  # fix for cluster_idx being a tuple
avg_distance = tf.reduce_mean(scores)
# Initialize the variables (i.e. assign their default value)
init_vars = tf.global_variables_initializer()


In [18]:
# Start TensorFlow session
sess = tf.Session()
sess.run(init_vars, feed_dict={X: full_data_x})
sess.run(init_op, feed_dict={X: full_data_x})
# Training
for i in range(1, num_steps + 1):
    _, d, idx = sess.run([train_op, avg_distance, cluster_idx], feed_dict={X: full_data_x})
    
    if i % 10 == 0 or i == 1:
        print("Step %i, Avg Distance: %f" % (i, d))
        print(idx)


Step 1, Avg Distance: 0.341471
[13 10  9 ...  9  3  4]


Step 10, Avg Distance: 0.221609
[13 10  9 ... 24  3  2]


Step 20, Avg Distance: 0.220328
[13 10  9 ... 24  3  2]


Step 30, Avg Distance: 0.219776
[13 10  9 ... 24  3  2]


Step 40, Avg Distance: 0.219419
[13 10  9 ... 24  3  2]


Step 50, Avg Distance: 0.219154
[13 10  9 ... 24  3  2]


In [20]:
# Assign a label to each centroid
# Count total number of labels per centroid, using the label of each training
# sample to their closest centroid (given by 'idx')
counts = np.zeros(shape=(k, num_classes))
for i in range(len(idx)):
    counts[idx[i]] += mnist.train.labels[i]
    # Assign the most frequent label to the centroid
    labels_map = [np.argmax(c) for c in counts]
    labels_map = tf.convert_to_tensor(labels_map)
# Evaluation ops
# Lookup: centroid_id -> label
cluster_label = tf.nn.embedding_lookup(labels_map, cluster_idx)
# Compute accuracy
correct_prediction = tf.equal(cluster_label, tf.cast(tf.argmax(Y, 1), tf.int32))
accuary_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# Test Model
test_x, test_y = mnist.test.images, mnist.test.labels
print("test accuary:", sess.run(accuary_op, feed_dict={X: test_x, Y: test_y}))


Tensor("embedding_lookup_1/Identity:0", dtype=int32)


test accuary: 0.7127
